In [31]:
from Code.LoadMnist import get_MNIST_data,get_load_data
from Code.def_NN import MLP
from Code.NNtraining import NN_training
from Code.FGSM import FGSM
import torch
import torchvision
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt

Load MNIST data

In [2]:
data_train,data_test = get_MNIST_data();
data_loader_train,data_loader_test = get_load_data(data_train,data_test,64)

Get multilayer perceptron

In [3]:
model = MLP()
cost = torch.nn.CrossEntropyLoss()

Train Neural Network

In [8]:
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.2 , momentum=0)
# n_epochs = 20

# model = NN_training(n_epochs,data_loader_train,data_loader_test,cost,model,optimizer,data_train,data_test)

# torch.save(model.state_dict(), "./model/model_parameter.pkl")

Load already-trainned Neural Network

In [9]:
model.load_state_dict(torch.load("./model/model_parameter.pkl"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Generating adversarial examples

In [175]:
data_loader_test = torch.utils.data.DataLoader(dataset=data_test,
                                          batch_size = 10,
                                          shuffle = True)
X_test, y_test = next(iter(data_loader_test))
X_test = FGSM(X_test,y_test,model,cost,0.1) #epsilon=0.1
Bild = X_test.detach().numpy()
for i in range(0,10):
    labels = './images/im'+np.str(i)+'.png'
    plt.imsave(labels,Bild[i,0],cmap ='gray')

Test accuracy of adversarial examples using trainned MLP

In [174]:
data_loader_test = torch.utils.data.DataLoader(dataset=data_test,
                                          batch_size = 64,
                                          shuffle = True)
sum_correct = 0;
for data in data_loader_test:
    X_test, y_test = data
    X_test = FGSM(X_test,y_test,model,cost,0.1) #epsilon=0.1
    inputs = Variable(X_test)
    outputs = model(inputs)
    _,pred = torch.max(outputs, 1)
    sum_correct += torch.sum(pred == y_test.data)

print("The accuracy of adversarial example is {:.3f}%".format(100*sum_correct/len(data_test)))

The accuracy of adversarial example is 13.000%
